In [31]:
import datetime
import os
import seaborn as sns
import sys
import time

%matplotlib inline

In [32]:
SCRIPTS_DIR = '../app'

sys.path.insert(1, SCRIPTS_DIR)

## Objective: New features to MA Trader 
### [1] Feat: MaxDrawdown as metric and NumExecution for information display [100%]
### [2] Feat: new trading strategy based upon double moving averages [100%]
### [3] Feat: new trading strategy w/ EMA [100%]
### [4] Feat: new trading strategy w/ MACD [100%]
### [5] Feat: visualization features

#### Prepare Data

In [33]:
# config

SECONDS_IN_ONE_DAY = 86400

STRATEGIES = ['MA-SELVES', 'DOUBLE-MA', 'MACD']
#STRATEGIES = ['MACD']

# simulation configuration
TOL_PCTS = [0.08, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
BUY_PCTS = [0.3, 0.4, 0.5, 0.6, 0.7]
SELL_PCTS = [0.3, 0.4, 0.5, 0.6, 0.7]

MA_LENGTHS = [3, 5, 10, 20, 30]
EMA_LENGTHS = [12, 26]

BUY_STAS = ('by_percentage')
SELL_STAS = ('by_percentage')

# currencies (digital + real-world)
CURS = ['BTC', 'ETH', 'LTC', 'BCH', 'ETC']
FIAT = ['USD']

# signals
NO_ACTION_SIGNAL = 'NO ACTION'
BUY_SIGNAL = 'BUY'
SELL_SIGNAL = 'SELL'

# epilson
EP_COIN = 10e-3
EP_CASH = 5

In [34]:
from trader_driver import TraderDriver
from util import load_csv

name,l = load_csv(csv_fn='../tests/fixtures/ETH_HISTORY.csv')

l = l[-300:]

In [35]:
init_amount = 3000
cur_coin = 5
mode = 'normal'

t_driver = TraderDriver(
    name=name,
    init_amount=init_amount,
    overall_stats=STRATEGIES,
    cur_coin=cur_coin,
    tol_pcts=TOL_PCTS,
    ma_lengths=MA_LENGTHS,
    ema_lengths=EMA_LENGTHS,
    buy_pcts=BUY_PCTS,
    sell_pcts=SELL_PCTS,
    buy_stas=BUY_STAS,
    sell_stas=SELL_STAS,
    mode=mode
)

t_driver.feed_data(data_stream=l)

info = t_driver.best_trader_info
# best trader
best_t = t_driver.traders[info['trader_index']]
# for a new price, find the trade signal
signal = best_t.trade_signal

signal

--> executed=feed_data using time=9.43 seconds.


{'action': 'NO ACTION', 'buy_percentage': 0.7, 'sell_percentage': 0.7}

In [36]:
best_t.all_history[:2]

[{'action': 'NO ACTION',
  'price': 132.0730737137,
  'date': '2019-12-31',
  'coin': 5,
  'cash': 3000,
  'portfolio': 3660.3653685685003},
 {'action': 'NO ACTION',
  'price': 128.7226035036,
  'date': '2020-01-01',
  'coin': 5,
  'cash': 3000,
  'portfolio': 3643.613017518}]

In [37]:
best_t.all_history_trade_only

[{'action': 'SELL',
  'price': 284.0004908975,
  'date': '2020-02-15',
  'coin': 1.5,
  'cash': 3974.121683778425,
  'portfolio': 4400.122420124675},
 {'action': 'BUY',
  'price': 127.2928961752,
  'date': '2020-03-13',
  'coin': 22.917121905371054,
  'cash': 1192.2365051335278,
  'portfolio': 4109.423324468327},
 {'action': 'BUY',
  'price': 124.97461472639999,
  'date': '2020-03-15',
  'coin': 29.461444880977954,
  'cash': 357.67095154005835,
  'portfolio': 4039.6036748233473},
 {'action': 'BUY',
  'price': 107.89834478510001,
  'date': '2020-03-17',
  'coin': 31.73545819642932,
  'cash': 107.30128546201752,
  'portfolio': 3531.5046958534763},
 {'action': 'BUY',
  'price': 107.89834478510001,
  'date': '2020-03-17',
  'coin': 32.41766219106473,
  'cash': 32.190385638605264,
  'portfolio': 3530.0024778570078}]

In [38]:
best_t.max_drawdown, best_t.num_transaction, best_t.high_strategy, best_t.num_buy_action, best_t.num_sell_action

(0.3286, 5, 'MA-SELVES', 4, 1)

In [39]:
info

{'buy_pct': 0.7,
 'sell_pct': 0.7,
 'tol_pct': 0.45,
 'buy': 'by_percentage',
 'sell': 'by_percentage',
 'init_value': 3660.3653685685003,
 'max_final_value': 12935.319,
 'rate_of_return': '253.389%',
 'baseline_rate_of_return': '36.329%',
 'coin_rate_of_return': '201.369%',
 'trader_index': 224}

In [40]:
best_t.crypto_prices[-10:]

[(393.1071990064, '2020-10-13'),
 (381.62561685459997, '2020-10-14'),
 (376.469523739, '2020-10-15'),
 (378.23951343650003, '2020-10-16'),
 (367.1522802856, '2020-10-17'),
 (368.23684389970003, '2020-10-18'),
 (376.5122024722, '2020-10-19'),
 (379.6604736902, '2020-10-20'),
 (368.8607245558, '2020-10-21'),
 (398.0277214424, '2020-10-22')]